In [8]:
import pandas as pd
from river import stream,tree,metrics
import utils
from encoding import prefix_bin
import numpy as np
from sklearn.metrics import classification_report, accuracy_score
import os
from tqdm import tqdm
import sliding_window
from sklearn.ensemble import RandomForestClassifier, IsolationForest
from sklearn.tree import DecisionTreeClassifier
import datetime, time
import importlib
importlib.reload(sliding_window)

import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
device = torch.device("cuda")
# Training
import torch.optim as optim

In [2]:
file_name = './data/loan_baseline.pnml_noise_0.15_iteration_1_seed_614_sample.csv'


dataset = stream.iter_csv(
            file_name
#             './data/loan_baseline.pnml_noise_0.15_iteration_1_seed_614_simple.csv',
            )

totallength = len(list(dataset))

In [3]:
dataset = stream.iter_csv(
            file_name,
            drop=['noise', 'lifecycle:transition', 'Variant', 'Variant index'],
            )
enctype = 'Index-base'

In [4]:
key_pair = {
'Case ID':'caseid',
'Activity':'activity',
# 'Resource':'resource',
'Complete Timestamp':'ts',
}
catatars= ['activity']#,'resource']

case_dict ={}
training_models ={}

casecount = 0
rowcounter = 0
resultdict ={}
acc_dict ={}
prefix_wise_window = {}
prediction_result = {}
graceperiod_finish=0
finishedcases = set()

In [5]:
# Sliding window for training setting
window_size = 100
retraining_size = 20
training_window = sliding_window.training_window(window_size,retraining_size)

In [10]:
class LSTM(nn.Module): # nn.Module 상속

    def __init__(self, inputs):
        super(LSTM, self).__init__()
        
        np_input = inputs.to_numpy()
        np_reshape = np.reshape(np_input, (np_input.shape[0],1,np_input.shape[1]))
        tensored_inputs = torch.tensor(np_reshape)
        self.input_size = tensored_inputs.shape[2]
        self.hidden_size =2* tensored_inputs.shape[2]
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=2, dropout=0.25, batch_first =False, bidirectional = False)
        self.drop = nn.Dropout(p=0.5)
        self.fc = nn.Linear(2*dimension, 1) #full connect

    def forward(self, text, text_len):
        text_emb = self.embedding(text)

        out_forward = output[range(len(output)), text_len - 1, :self.dimension]
        out_reverse = output[:, 0, self.dimension:]
        out_reduced = torch.cat((out_forward, out_reverse), 1) #[range(len(output)), text_len -1 + 0 , self.dimension]
        text_fea = self.drop(out_reduced)

        text_fea = self.fc(text_fea)
        text_fea = torch.squeeze(text_fea, 1)
        text_out = torch.sigmoid(text_fea)

        return text_out

In [ ]:
class LSTMGenerator(nn.Module):
    def __init__(self, seq_len, input_size, batch, hidden_size , num_layers, num_directions):
        super().__init__()
        self.input_size = input_size
        self.h = torch.randn(num_layers * num_directions ,batch, hidden_size)
        self.c = torch.randn(num_layers * num_directions ,batch, hidden_size)

        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, dropout=0.25, batch_first =True, bidirectional = False)


        latent_vector_size =50 * batch
        self.linear1 = nn.Linear(batch * seq_len *hidden_size, latent_vector_size)
        # self.linear2 = nn.Linear(latent_vector_size,batch*seq_len*hidden_size)
        self.linearHC = nn.Linear(num_layers *hidden_size *batch, latent_vector_size)
        # self.linearHCO = nn.Linear(3*latent_vector_size,batch*seq_len*hidden_size )
        self.linearHCO = nn.Linear( 3 *latent_vector_size ,batch *seq_len *input_size )




        # Define sigmoid activation and softmax output
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        self.softmax = nn.Softmax()
        
    def forward(self, x):
        # x = x.view((1,x.size()[0], x.size()[1]))
        # Pass the input tensor through each of our operations
        # print("inputsize:", x.size())
        output, (h ,c) = self.lstm(x, (self.h, self.c))
        # print("inputsize:", x.size(),"output size:", output.size())
        # print("h size:", h.size(),"c size:", c.size())
        self.h = h.detach()
        self.c = c.detach()

        # Executing Fully connected network
        # print("The size of output:", output.size(), h.size(), c.size())
        u = output.reshape((output.size()[0 ] *output.size()[1 ] *output.size()[2]))
        u = self.relu(self.linear1(u))
        # print("The size of lninera1:", u.size())
        # u = self.linear2(u)

        # Flating h and feeding it into a linear layer
        uH = F.leaky_relu(self.linearHC(h.reshape((h.size()[0 ] *h.size()[1 ] *h.size()[2]))))
        uC = F.leaky_relu(self.linearHC(c.reshape((c.size()[0 ] *c.size()[1 ] *c.size()[2]))))
        uHCO = torch.cat((uH ,uC ,u))
        uHCO = self.linearHCO(uHCO)
        u= uHCO

        # output = u.view((output.size()[0],output.size()[1],output.size()[2]))
        output = u.view((output.size()[0],output.size()[1],self.input_size))
        # print("output size finally:", output.size())


        return output

In [6]:
def display_progress(row_counting, total_length, interval=500):
    if rowcounter%interval == 0:
        print(round(rowcounter*100/totallength,2) ,'%', 'Case finished: %s'%(casecount), 'Running cases: %s'%(len(case_dict)))

In [11]:
start_time = time.time()

for x,y in dataset:
    display_progress(rowcounter, totallength)
    rowcounter +=1
    
    utils.dictkey_chg(x, key_pair)
    # Event stream change dictionary keys
    x['ts'] = x['ts'][:-4]
    
    # Check label possible
    
    # Initialize case by prefix length
    caseid = x['caseid']
    x.pop('caseid')
    
    case_bin = prefix_bin(caseid, x)
    
    if caseid not in list(case_dict.keys()):
        case_dict[caseid] = []
        case_bin.set_prefix_length(1)
        
    elif caseid in finishedcases:
        continue
    
    else:
        case_bin.set_prefix_length(len(case_dict[caseid])+1)
        case_bin.set_prev_enc(case_dict[caseid][-1])
    
    # Encode event and cases and add to DB
    ts = case_bin.event['ts']
    case_bin.update_encoded(catattrs=catatars,enctype=enctype)
    
    # Set current activity as outcome of previous event
    if case_bin.prefix_length != 1:
        case_bin.prev_enc.update_truelabel(x['activity'])

    # First prediction for current event
    
    last_event = case_bin
    modelid = 'None'
    prediction = 'Not Available'

    if len(training_window.getAllitems()) !=0:
        if 'window_%s'%(last_event.prefix_length) in list(prefix_wise_window.keys()) and 'detector_window_%s'%(last_event.prefix_length) in training_models.keys():
            modelid, prediction = predict_activity_proba(last_event)
#             feature_matrix = prefix_wise_window['window_%s'%(last_event.prefix_length)][0].columns.values
#             current_event = utils.readjustment_training(last_event.encoded, feature_matrix)
#             current_event = pd.Series(current_event).to_frame().T
#             prediction = [training_models['detector_window_%s'%(last_event.prefix_length)][1].predict_proba(current_event), training_models['detector_window_%s'%(last_event.prefix_length)][1].classes_]
#             modelid = training_models['detector_window_%s'%(last_event.prefix_length)][0]
    case_bin.update_prediction((modelid, (prediction,ts)))        
            
    # Update training window and finish the case
    if x['activity'] == 'End':
        training_window.update_window({caseid: case_dict[caseid]})        
        if training_window.retraining == training_window.retraining_count:            
            training_models = training_stage(training_window, training_models)
            prefix_wise_window = training_window.prefix_wise_window()
            
        resultdict[caseid] = case_dict[caseid]
        case_dict.pop(caseid)

        casecount +=1
        for x in case_dict:
            last_event = case_dict[x][-1]
            modelid = 'None'
            prediction = 'Not Available'

            if len(training_window.getAllitems()) !=0:
                prefix_wise_window = training_window.prefix_wise_window()
                if 'window_%s'%(last_event.prefix_length) in list(prefix_wise_window.keys()) and 'detector_window_%s'%(last_event.prefix_length) in training_models.keys():
                    modelid, prediction = predict_activity_proba(last_event)

#                     feature_matrix = prefix_wise_window['window_%s'%(last_event.prefix_length)][0].columns.values
#                     current_event = utils.readjustment_training(last_event.encoded, feature_matrix)
#                     current_event = pd.Series(current_event).to_frame().T
#                     prediction = [training_models['detector_window_%s'%(last_event.prefix_length)][1].predict_proba(current_event), training_models['detector_window_%s'%(last_event.prefix_length)][1].classes_]
#                     modelid = training_models['detector_window_%s'%(last_event.prefix_length)][0]
            case_dict[x][-1].update_prediction((modelid, (prediction,ts)))        
        training_window.reset_retraining_count()
    else:
        case_dict[caseid].append(case_bin)


end_time = time.time()

0.0 % Case finished: 0 Running cases: 0


NameError: name 'training_stage' is not defined

In [25]:
pw_window = training_window.prefix_wise_window()
for x in pw_window:
#     print(pw_window[x])
    print(pw_window[x][0])
    break
#     print(len(pw_window[x][1]))

    duration_1  cumduration_1  activity_1 Start
0            0              0                 1
1            0              0                 1
2            0              0                 1
3            0              0                 1
4            0              0                 1
5            0              0                 1
6            0              0                 1
7            0              0                 1
8            0              0                 1
9            0              0                 1
10           0              0                 1
11           0              0                 1
12           0              0                 1
13           0              0                 1
14           0              0                 1
15           0              0                 1
16           0              0                 1
17           0              0                 1
18           0              0                 1
19           0              0           